In [1]:
from datasets import load_dataset
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np

In [2]:
dataset = load_dataset("dzunggg/legal-qa-v1")
train = dataset["train"]
df = pd.DataFrame(train)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: a490aac1-d894-4e8b-848f-fedbdf1fed5f)')' thrown while requesting HEAD https://huggingface.co/datasets/dzunggg/legal-qa-v1/resolve/6280beb74faf5b4dfd1f63adbf7d18908b377b93/legal-qa-v1.py
Retrying in 1s [Retry 1/5].


legal_qa_full.csv: 0.00B [00:00, ?B/s]

d:\Mas\anaconda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Inspiron 5584\.cache\huggingface\hub\datasets--dzunggg--legal-qa-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Generating train split:   0%|          | 0/3742 [00:00<?, ? examples/s]

In [3]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_question"] = df["question"].apply(preprocess)

In [4]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = model.encode(df["clean_question"].tolist(), show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

d:\Mas\anaconda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Inspiron 5584\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/117 [00:00<?, ?it/s]

In [5]:
NUM_CLUSTERS = 6  

kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
clusters = kmeans.fit_predict(embeddings)

df["cluster"] = clusters

In [8]:
cluster_samples = {}
for c in range(NUM_CLUSTERS):
    questions = df[df["cluster"] == c]["question"].head(8).tolist()
    cluster_samples[c] = questions

In [11]:
from openai import OpenAI
client = OpenAI(
    api_key="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",
    base_url="https://openrouter.ai/api/v1"
)
def name_cluster(questions):
    prompt = f"""
    You are an expert in legal domain classification.

    Based on the following sample legal questions, determine the best category name.
    Keep the category name short (2–4 words).
    Questions:
    {questions}

    Return only the name of the category.
    """

    response = client.chat.completions.create(
        model="meta-llama/llama-3-70b-instruct",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content.strip()

cluster_names = {}
for c in range(NUM_CLUSTERS):
    q_list = "\n".join(cluster_samples[c])
    cluster_names[c] = name_cluster(q_list)


In [12]:
df["cluster_name"] = df["cluster"].map(cluster_names)

df[["question", "cluster", "cluster_name"]].head(20)

,question,cluster,cluster_name
0,Q: I was wondering if a pain management office...,4,Civil Law Disputes
1,Q: My houseboat was moved while I was at work ...,0,Landlord-Tenant Disputes
2,Q: Wondering if I could sue a dealership if I’...,1,Civil Disputes
3,Q: Do I need to file for CS & custody through ...,2,Family Law
4,"Q: I am planning on releasing a tabletop rpg ""...",5,Legislation and Compliance
5,Q: How can I communicate intent to file crimin...,2,Family Law
6,Q: I have sole physical / legal custody of my ...,2,Family Law
7,Q: Letter from Board of Supervisor says I have...,1,Civil Disputes
8,"Q: Hello, (Ref: Maritime PIRACY law vs. Civil/...",3,Criminal Law
9,Q: Can I move out of state with my children if...,2,Family Law
